<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_13_3_speech2text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 13: Speech Processing**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 13 Material

Module 13: Speech Processing

* Part 13.1: Intro to Speech Processing [[Video]](https://www.youtube.com/watch?v=ILNcv9zrMyQ&ab_channel=JeffHeaton) [[Notebook]](t81_559_class_13_1_speech_models.ipynb)
* Part 13.2: Text to Speech [[Video]](https://www.youtube.com/watch?v=O5_oaK5fHqI&ab_channel=JeffHeaton) [[Notebook]](t81_559_class_13_2_text2speech.ipynb)
* **Part 13.3: Speech to Text** [[Video]](https://www.youtube.com/watch?v=zor64w90fpQ&ab_channel=JeffHeaton) [[Notebook]](t81_559_class_13_3_speech2text.ipynb)
* Part 13.4: Speech Bot [[Video]](https://www.youtube.com/watch?v=8fgxX6yLorI&ab_channel=JeffHeaton) [[Notebook]](t81_559_class_13_4_speechbot.ipynb)
* Part 13.5: Future Directions in GenAI [[Video]](https://www.youtube.com/watch?v=T4AYP_XXTbg&ab_channel=JeffHeaton) [[Notebook]](t81_559_class_13_5_future.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !sudo apt-get install portaudio19-dev python3-pyaudio
    !pip install langchain langchain_openai openai pyaudio sounddevice numpy pydub

# Part 13.3: Speech to Text

In this module, we'll delve into the realm of speech-to-text technology, focusing on the powerful capabilities offered by OpenAI's models. Speech-to-text, also known as automatic speech recognition (ASR), is a technology that converts spoken language into written text. OpenAI's speech-to-text models represent the cutting edge of this field, leveraging advanced machine learning techniques to achieve high accuracy and robustness across various accents, languages, and acoustic environments. We'll explore how these models can be integrated into applications to enable voice-based interactions, transcription services, and accessibility features. By harnessing OpenAI's speech-to-text technology, we'll unlock new possibilities for human-computer interaction and demonstrate how to transform audio input into actionable text data with remarkable precision.


Note we will make use of the technique described here to record audio in CoLab.

https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be




In [ ]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import io
from IPython.display import Audio

from pydub import AudioSegment

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
    const reader = new FileReader()
    reader.onloadend = e => resolve(e.srcElement.result)
    reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true })
    recorder = new MediaRecorder(stream)
    chunks = []
    recorder.ondataavailable = e => chunks.push(e.data)
    recorder.start()
    await sleep(time)
    recorder.onstop = async ()=>{
        blob = new Blob(chunks)
        text = await b2text(blob)
        resolve(text)
    }
    recorder.stop()
})
"""

The following code uses JavaScript to record audio for a specified amount of time.

In [ ]:
def record(seconds=3):
    print(f"Recording now for {seconds} seconds.")
    display(Javascript(RECORD))
    s = output.eval_js('record(%d)' % (seconds * 1000))
    binary = b64decode(s.split(',')[1])

    # Convert to AudioSegment
    audio = AudioSegment.from_file(io.BytesIO(binary), format="webm")

    # Export as WAV
    audio.export("recorded_audio.wav", format="wav")
    print("Recording done.")
    return audio

The following code provides a quick test of this technique.

In [ ]:
# Record 5 seconds of audio
audio = record(5)

print("Recording complete. Audio saved as 'recorded_audio.wav'")

# Play back the recorded audio
display(Audio("recorded_audio.wav",autoplay=True))

This code snippet demonstrates how to use OpenAI's speech-to-text API to transcribe audio files. It defines a function transcribe_audio that takes a filename as input. The function opens the specified audio file in binary mode and uses the OpenAI client to create a transcription. The client.audio.transcriptions.create() method is called with two parameters: the model ("whisper-1") and the audio file. Whisper is OpenAI's state-of-the-art speech recognition model, known for its robustness across various languages and accents. The function returns the transcribed text. In the example usage, an audio file named "recorded_audio.wav" is transcribed, and the resulting text is printed. This code provides a simple yet powerful way to convert speech to text, which can be invaluable for tasks such as generating subtitles, creating searchable archives of audio content, or enabling voice commands in applications.

In [ ]:
from openai import OpenAI

client = OpenAI()

def transcribe_audio(filename):
    with open(filename, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
    return transcription.text

# Transcribe the recorded audio
transcription = transcribe_audio("recorded_audio.wav")
print("Transcription:")
print(transcription)

### Voice Change
This code snippet demonstrates a complete workflow for speech-to-text and text-to-speech conversion using OpenAI's APIs. The generate_text function uses OpenAI's TTS-1 model with the "nova" voice to convert text into speech, returning the raw audio data. The speak_text function builds upon this by taking a text input, generating the corresponding audio, and then playing it using IPython's Audio display function with autoplay enabled. The main workflow begins by recording audio for 5 seconds (using a record function not shown in the snippet), transcribing it using the previously defined transcribe_audio function, printing the transcription, and finally speaking the transcribed text back using the speak_text function. This creates a full circle of voice interaction: recording speech, converting it to text, and then converting that text back into speech, effectively demonstrating both speech recognition and speech synthesis capabilities in a single, cohesive process.

In [ ]:


def generate_text(text):
    response = client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=text
    )
    audio_data = response.content
    return audio_data  # Return the audio data directly

def speak_text(text):
    audio_data = generate_text(text)
    display(Audio(audio_data, autoplay=True))

# Transcribe the recorded audio
audio = record(5)
transcription = transcribe_audio("recorded_audio.wav")
print("Transcription:")
print(transcription)
speak_text(transcription)